## Eksploracyjna analiza danych - temat 5

**Temat:**
*„Mamy co prawda dodatkowe benefity dla naszych najlepszych klientów, ale może dałoby się ustalić kto potencjalnie jest skłonny wydawać u nas więcej?.”*

> Celem jest identyfikacja na podstawie historycznych zakupów, czy w następnych okresie klient kupi więcej.



**Analiza zadania:**

*   co to znaczy klient kupi "więcej"?  

1.   w następnym analizowanym miesiącu klient kupi więcej o *X* zł (wstępnie 100zł ale jeszcze do uzgodnienia z zamawiającym) niż średnia wydatków z wszystkich sesji,  które klient w okresie historycznym odbył? 
2.   a może klient zapłaci o *X* zł więcej od tego co średnio wydawał w miesiącu (na podstawie danych historycznych)

Pytanie czy w przypadku pkt. 1 i 2, należy analizować także sesje nie zakończone zakupem?
Wszystkie trzy zagadnienia można potraktować jako zadanie klasyfikacji.

Alternatywnie w zadaniu regresji można byłoby przewidywać całkowitą sumę wydaną w kolejnym okresie, albo różnicę pomiędzy średnimi historycznymi wydatkami a przyszłymi.

Innym pomysłem niż analizowanie wydawanych kwot jest operowanie na liczbie zakupów. Można wówczas przewidywać czy w następnym analizowanym okresie, klient kupi produkt (klasyfikacja), albo ile ich ich kupi (regresja). 
Jednak takie podejście jest mniej informatywne dla zamawiającego.



*   Doprecyzowania z zamawiającym wymaga także kwestia jak długi horyzont danych historycznych możemy wykorzystać do modelowania i na ile miesięcy na przyszłość mamy dokonywać predykcji



Sugerujemy, że najlepszym rozwiązaniem będzie wykonanie **zadania klasyfikacji, gdzie przewidywać będziemy czy klient w przyszłości zrobi zakupy o *X* zł większe niż średnio w przeszłości**. Szczegóły zaznaczone powyżej dotyczące np. horyzontu czasu czy kwestii uwzględniania bądź nie sesji niezakończonych zakupem, zostaną omówione na kolejnym spotkaniu z zamawiającym.

Wówczas też nastąpi przekazanie dodatkowych danych.

**Kryteria sukcesu**

Celem minimum jest stworzenie klasyfikatora, który będzie lepszy niż losowy. 

**Metryka biznesowa**
Skuteczność modelu powinna wynieść więcej niż 50%. 

**Metryka analityczna**
W ramach modelowania analizowana będzie zbalansowana skuteczność ("balanced accuracy") oraz AUC.

Poniższe analizy zostały wykonane na podstawie pierwotnie dostarczonego zbioru danych.
Warto też nadmienić, że podczas spotkania otrzymaliśmy informację, że udzielane dotychczas rabaty mają charakter losowy.

# Część analityczna

Import pakietów

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

Wczytanie pliku *sessions* oraz analiza podstawowych charakterystyk.

In [ ]:
session_df = pd.read_json('/content/drive/My Drive/IUM/data/sessions.jsonl', lines=True)

print(session_df.info(), end='\n\n')
session_df.head()

ValueError: ignored

In [ ]:
session_df['event_type']=session_df['event_type'].map({'VIEW_PRODUCT': 0 , 'BUY_PRODUCT':1})

In [ ]:
product_df = pd.read_json('/content/drive/My Drive/IUM/data/products.jsonl', lines=True)

print(product_df.info(), end='\n\n')
product_df.head()

In [ ]:
df=session_df.merge(product_df, how='inner', on='product_id')

In [ ]:
df.columns

In [ ]:
df=df[['session_id', 'timestamp', 'user_id', 'product_id', 'event_type',
       'offered_discount', 'price']]

In [ ]:
df['timestamp']=pd.to_datetime(df.timestamp)
df['month'] = df['timestamp'].dt.month

In [ ]:
df

In [ ]:
product_df[product_df.product_id==1544]  

####### uwaga - niektóre produkty mają ujemne ceny, a niektóre są ogromne - trzeba sprawdzić ile takich jest
###### czy jednostki są te same? - pytanie do klienta


In [ ]:
def f(event_type, price):
    if event_type==1:
        return price
    else: 
        return 0
    

df['purchase']= df.apply(lambda x: f(x.event_type, x.price), axis=1)
# drop 'price' column

In [ ]:
df

In [ ]:
# gdy będzie więcej danych zrobimy jeszcze zbiór val
df_train = df[df.month!=3]
df_test = df[df.month==3]

In [ ]:
df.loc[(df.user_id==102.0) & (df.event_type==1)]

In [ ]:
#### przykładowy klient
d102=df_train[df_train['user_id']==102]
d102

In [ ]:
train_users_total_sum=df_train.groupby('user_id')['purchase'].sum()
train_users_mean_discount=df_train.groupby('user_id')['offered_discount'].mean()
train_users_corr_purchase_discount=df_train.groupby('user_id').apply(lambda x: np.corrcoef(x.purchase, x.offered_discount)[0][1])

In [ ]:
# https://stackoverflow.com/questions/55649356/how-can-i-detect-if-trend-is-increasing-or-decreasing-in-time-series

def trenddetector(array_of_data, order=1):
    array_of_data=list(array_of_data[array_of_data!=0])

    if len(array_of_data)<2:
        return 0
    else:
        result = np.polyfit(np.arange(len(array_of_data)), list(array_of_data), order)
        slope = result[-2]
        return float(slope)

In [ ]:
train_users_purchase_trend=df_train.groupby('user_id')['purchase'].apply(lambda x: trenddetector(x))

In [ ]:
df_train_users=pd.DataFrame({'total_sum': train_users_total_sum, 
                             'mean_discount': train_users_mean_discount, 
                             'corr_purchase_discount': train_users_corr_purchase_discount,
                            'spending_trend': train_users_purchase_trend})


# train_users_purchase_trend - wartości mają duży rozrzut, 
# więc potrzebna będzie normalizacja, albo jednak rezygnacja z tej zmiennej

In [ ]:
df_train_users
### rozumiemy, że w żaden sposób nie możemy używać user_id, tak aby rozwiązanie było jak najbardziej ogólne

In [ ]:
# w tabeli powyżej trzeba dokleić info o wydanej sumie w kolejnym miesiącu
# może warto dodać info o znaczniku czasu

Trzeba przeanalizować ceny produktów

In [ ]:
plt.hist(df.purchase)  ### ile klienci wydali w ciągu ostatnich 2 miesięcy

In [ ]:
plt.hist(np.log1p(df.purchase)) #zlogarytmowane wydatki miesięczne klientów

# TRZEBA SPRAWDZIĆ CZY COŚ ZE STARSZYCH ANALIZ MOŻE SIĘ PRZYDAĆ

# Dodatkowe analizy eksploracyjne

Ile czynności ooglądania produktu na stronie zakończyło się zakupem?

In [ ]:
ax = sns.countplot(x="event_type", data=data_df)

Ile sesji skończyły się zakupem towaru na każdego użytkownika (sessions per user)?

In [ ]:
sessions_count = pd.DataFrame(session_df[session_df["event_type"] == 1]).groupby('user_id').count()["purchase_id"]
sessions_count = sessions_count.reset_index().rename(columns={"purchase_id": 'successfull_sessions'})

sessions_count

users_data = pd.read_json('data/users.jsonl', lines=True)

data_per_user = pd.merge(sessions_count, users_data, on='user_id', how='outer')
data_per_user['successfull_sessions'].fillna(0, inplace=True)
data_per_user

Ile zapłacił pojedyńczy klient uwzględniając zniżki?

In [ ]:
sessions_length = len(session_df)

successful_sessions = session_df[session_df["event_type"] == 1]

prices = pd.merge(successful_sessions, product_df, on='product_id', how='inner')
prices['price_incl_discount'] = prices['price'] - (prices['price'] * (prices['offered_discount'] / 100))

#prices.price.hist()

money_spent_per_user = prices.groupby('user_id').sum()['price_incl_discount']
money_spent_per_user = pd.DataFrame(money_spent_per_user)
money_spent_per_user['user_id'] = money_spent_per_user.index
money_spent_per_user.reset_index(drop=True, inplace=True)

Ile dni minęło od ostatniego zakupu klienta?

In [ ]:
time_diff = session_df[['user_id', 'timestamp', 'event_type']]
bought_time_diff = session_df[session_df.event_type == 1][['user_id', 'timestamp', 'event_type']]
bought_time_diff.reset_index(drop=True, inplace=True)
bought_time_diff

last_purchase_by_client = bought_time_diff.groupby('user_id').max()
last_purchase_by_client.columns = ['last_purchase_time', 'event_type'] 
last_purchase_by_client['now'] = pd.Timestamp.now()

no_bought_time = (last_purchase_by_client.now - last_purchase_by_client.last_purchase_time).dt.days
no_bought_time = pd.DataFrame(no_bought_time)
no_bought_time.columns = ['days_after_last_purchase']

no_bought_time

Częstotliwość zakupów przez klienta

In [ ]:
first_purchase_by_client = bought_time_diff.groupby('user_id').min()

first_purchase_by_client.columns = ['first_purchase_time', 'event_type']
first_purchase_by_client

purchases = pd.DataFrame((last_purchase_by_client.last_purchase_time - first_purchase_by_client.first_purchase_time).dt.days)
purchases.columns = ['time_between_first_and_last_purchase']

purchases['time_between_first_purchase_and_now'] = (pd.Timestamp.now() - first_purchase_by_client.first_purchase_time).dt.days
purchases['number_of_purchases'] = bought_time_diff.groupby('user_id').count().timestamp
purchases['purchase_frequency_by_user_between_first_and_last_per_day'] = purchases['number_of_purchases'] / purchases['time_between_first_and_last_purchase'] 
purchases['purchase_frequency_by_user_between_first_and_now_per_day'] = purchases['number_of_purchases'] / purchases['time_between_first_purchase_and_now']

purchases

Ile razy produktów w ramach sesji uczestnicy oglądali?

In [ ]:
session_product=data_df.groupby(['session_id'])['product_id'].count()
plt.hist(session_product)

In [ ]:
session_results=data_df.groupby(['session_id'])['event_type'].min()  #### to check whether it works exactly how I would like it to work

In [ ]:
session_prod_event=pd.DataFrame(session_product).join(pd.DataFrame(session_results))
session_prod_event.sort_values(by=['product_id'], inplace=True)  ## it would be good to change column names

### tutaj gdy product_id==0 to oznacza, że w oryginalnych danych było NaN

In [ ]:
session_prod_event['event_type']=session_prod_event['event_type'].map({'VIEW_PRODUCT':0, 'BUY_PRODUCT':1})

In [ ]:
n_prod_buy=session_prod_event.groupby(['product_id'])['event_type'].mean()
plt.bar(n_prod_buy.index, n_prod_buy)
plt.xlabel('liczba oglądanych produktów')
plt.ylabel('prawdopodobieństwo zakupu')

Wstępna analiza pokazuje, że istnieje zależność, że im więcej ktoś przejrzał produktów, tym szansa, że coś kupi wzrasta.

In [ ]:
data_df

Długość trwania dostawy do klientów

In [ ]:
delivery_df = pd.read_json('data/deliveries.jsonl', lines=True)

delivery_sessions = pd.merge(session_df, delivery_df, on='purchase_id', how='inner')

fmt = '%Y-%m-%dT%H:%M:%S'
delivery_sessions['purchase_timestamp'] = pd.to_datetime(delivery_sessions['purchase_timestamp'], format=fmt, errors='coerce')
delivery_sessions['delivery_timestamp'] = pd.to_datetime(delivery_sessions['delivery_timestamp'], format=fmt, errors='coerce')

delivery_sessions['delivery_time'] = (delivery_sessions.delivery_timestamp - delivery_sessions.purchase_timestamp)
delivery_sessions[['user_id', 'purchase_timestamp', 'delivery_timestamp', 'delivery_time']] 

Top 10 klientów według wydanych pieniędzy

In [ ]:
users.sort_values('price_incl_discount', ascending=False).head(10)

Top 10 klientów według kupionych rzeczy

In [ ]:
users.sort_values('successfull_sessions', ascending=False).head(10)

## Analiza czasu trwania sesji

In [ ]:
diff_time=data_df.groupby(['session_id'])['timestamp'].max()-data_df.groupby(['session_id'])['timestamp'].min()

In [ ]:
d=pd.DataFrame(diff_time)
d=d.reset_index()
d.columns=['session_id', 'time_diff']

### w ramce danych do modelowania należy dać informację, ile całkowicie trwała sesja oraz ile produktów obejrzano

In [ ]:
d=d.sort_values(['time_diff'])
d.index=d.session_id

In [ ]:
session_results=session_results.map({'VIEW_PRODUCT':0, 'BUY_PRODUCT':1})

In [ ]:
d=d.join(session_results)
d.drop('session_id', axis=1, inplace=True)

In [ ]:
d

In [ ]:
d['time_diff'] = d['time_diff'].apply(lambda x: x.seconds)

In [ ]:
plt.hist(d['time_diff'])
plt.xlabel('czas trwania sesji')

In [ ]:
## Warto byłoby dodać wizualizację jaka jest relacja między czasem trwania sesji a zakupem 

## Dodatkowe analizy

Ile razy dany produkt był kupiony?

In [ ]:
plt.figure(figsize=(10,10))
sns.countplot(data_df[data_df.event_type=='BUY_PRODUCT'].groupby('product_id')['product_id'].count())

Jaki jest najwyższy współczynnik liczby sprzedaży danego produktu liczby jego oglądania?

In [ ]:
df_tmp.groupby('product_id')['event_type'].mean().max()

# Braki w danych

In [ ]:
3097/44345 

Wniosek: Braki występują w kolumnach: *used_id*, *product_id* oraz *purchase_id*. Było to uwidocznione w pierwsze analizie po wczytaniu danych. Szczególnie dużo jest ich w ostatnio wymienionej kolumnie, gdzie tylko ok. 7% rekordów ma zdefiniowaną wartość. W przypadku pozostałych kolumn braki stanowią jedynie ok. 5\%.

Zweryfikowano czy duża liczba braków w kolumnie *purchase_id* wynika z tego, że niewielka liczba sesji polegających na oglądaniu konkretnego produktu zakończyła się zakupem. 

In [ ]:
df_view_prod=data_df[data_df['event_type']=='VIEW_PRODUCT']

np.sum(np.isnan(df_view_prod.purchase_id))/len(df_view_prod)

Okazuje się, że brak wartości w kolumnie *purchase_id* wynika z braku zakupu. Braki więc nie mają charakteru losowego, a wynikają z charakteru i logiki danych. 

W tej sytuacji można byłoby zamienić braki na wartość neutralną, czyli taką jaka nie występuje w kolumnie *purchase_id* - np. 0 albo -1.

In [ ]:
data_df[data_df.user_id.isnull()]

In [ ]:
data_df[data_df.product_id.isnull()]

Nie widać jasnego powodu dlaczego w ok. 5% przypadków występuje brak w kolumnie *user_id* i *product_id*, dlatego też nie podejrzewa się informatywnego błędu systematycznego. **W tej sytuacji wiersze z brakami w tymi kolumnach podda się usunięciu.**

In [ ]:
data_df=data_df[data_df.user_id.notnull()]
data_df=data_df[data_df.product_id.notnull()]
data_df.shape

Można by było też zamienić na jakieś wartości, ale skoro braków jest mało, to chyba mała strata.

Liczba użytkowników:

In [ ]:
len(np.unique(data_df.user_id))

Liczba sesji:

In [ ]:
len(np.unique(data_df.session_id))

Liczba produktów:

In [ ]:
len(np.unique(data_df.product_id))

In [ ]:
ax = sns.countplot(x="offered_discount", data=data_df)

# Time analysis

There is no information about time zone therefore we assume that everything is in the same timezone.

In [ ]:
data_df['timestamp']=pd.to_datetime(data_df.timestamp)

In [ ]:
data_df['date']=data_df['timestamp'].dt.date

In [ ]:
data_df['time']=data_df['timestamp'].dt.time
data_df['day']=data_df['timestamp'].dt.day
data_df['month']=data_df['timestamp'].dt.month
data_df['year']=data_df['timestamp'].dt.year

#data_df.drop('timestamp', axis=1, inplace=True)

In [ ]:
np.unique(data_df.year)

Wszystkie rekordy są z tego samego roku, więc informacja o roku nic nie wnosi i można ją usunąć.

In [ ]:
np.unique(data_df.month)

Informacje są tylko z pierwszego kwartału 2022r. 

In [ ]:
print(pd.date_range(
  start="2022-01-01", end="2022-03-31").difference(data_df.date))

Okazuje się, że informacja nie jest z całych trzech miesięcy, tj. kończy się w dniu 28-03-2022.

In [ ]:
data_df['timestamp'][0].strftime("%w")  # day of week  -- transform in such a way to df

In [ ]:
# we should have both info about day of week and day of month

In [ ]:
data_df['hour']=data_df['timestamp'].dt.hour

In [ ]:
### hour, day of week, day of month - we will cyclic feature encoding

![image.png](attachment:image.png)

https://www.kaggle.com/code/avanwyk/encoding-cyclical-features-for-deep-learning/notebook

In [ ]:
data_df['hour_sin'] = np.sin(2 * np.pi * data_df['hour']/23.0)
data_df['hour_cos'] = np.cos(2 * np.pi * data_df['hour']/23.0)

In [ ]:
data_df

# Analiza zniżek

In [ ]:
plt.hist(data_df['offered_discount'], bins=20)

In [ ]:
data_df['event_type']=data_df['event_type'].map({'VIEW_PRODUCT':0, 'BUY_PRODUCT':1})
discount_buy=data_df.groupby(['offered_discount'])['event_type'].mean()
plt.bar(discount_buy.index, discount_buy)

In [ ]:
discount_user=data_df.groupby(['user_id'])['offered_discount'].mean()
plt.hist(discount_user)